In [6]:
import math

In [7]:
import diamond as Aztec # the (rather complicated) data structure

In [8]:
import tiling # this contains the algorithm(s) i.e. for tiling and the height function 

In [9]:
sizes = [25, 50, 75, 100]

In [13]:
def Z(x,y) :
    (2 / math.pi) [ (x - 1/2) * math.arctan( (math.sqrt(1/4 - (x-1/2)**2 - (y - 1/2)**2)) / (1/2 - y) )
                   + (1/2)* math.arctan( (2*(x - 1/2)*(1/2 - y)) / (math.sqrt(1/4 - (x - 1/2)**2 - (y - 1/2)**2)) ) 
                   - (1/2 - y)* math.arctan( (x - 1/2) / math.sqrt(1/4 - (x - 1/2)**2 - (y - 1/2)**2 ) )]

In [15]:
def f_star(y, x) :
    if (y > 0 and y < 1 / 2) :
        if(x >= 0 and x <= (1 - 2*math.sqrt(y*(1-y))) / 2) :
            return 0
        elif (x > ((1 - 2*math.sqrt(y*(1-y))) / 2 ) and x < ((1 + 2*math.sqrt(y*(1-y))) / 2 )) :
            return (y/2)  + (1/2)*Z(x,y)
        elif (x >= ((1 + 2*math.sqrt(y*(1-y))) / 2 )) :
            return y
    elif (y == 1/2) :
        return x / 2
    else :
        return x - f_star(1-y, x)